In [1]:
import json
import pandas as pd

In [2]:
# Your annotations csv file needs to have the following columns:

column_names =['filename', 'class', 'width', 'height', 'xmin', 'ymin', 'xmax','ymax']

In [3]:
df = pd.read_csv('annotations.csv')
df['class'] = ['TBbacillus']*df.shape[0]
df = df.rename(columns = {'img_name': 'filename'})
df['width'] = df['xmax'] - df['xmin']
df['height'] = df['ymax'] - df['ymin']
df.to_csv('annotations_edited.csv', index=False)

df

,filename,xmin,xmax,ymin,ymax,shape,class,width,height
0,_tuberculosis-phone-0001.jpg,830,886,1029,1093,"(1224, 1632, 3)",TBbacillus,56,64
1,_tuberculosis-phone-0001.jpg,991,1088,765,810,"(1224, 1632, 3)",TBbacillus,97,45
2,_tuberculosis-phone-0001.jpg,1002,1066,397,494,"(1224, 1632, 3)",TBbacillus,64,97
3,_tuberculosis-phone-0001.jpg,1448,1487,843,928,"(1224, 1632, 3)",TBbacillus,39,85
4,_tuberculosis-phone-0001.jpg,1185,1241,579,670,"(1224, 1632, 3)",TBbacillus,56,91
...,...,...,...,...,...,...,...,...,...
9964,_tuberculosis-phone-1264.jpg,1309,1342,820,874,"(1224, 1632, 3)",TBbacillus,33,54
9965,_tuberculosis-phone-1264.jpg,1446,1477,224,282,"(1224, 1632, 3)",TBbacillus,31,58
9966,_tuberculosis-phone-1265.jpg,1101,1134,273,370,"(1224, 1632, 3)",TBbacillus,33,97
9967,_tuberculosis-phone-1265.jpg,938,973,552,618,"(1224, 1632, 3)",TBbacillus,35,66


In [5]:
path = 'annotations_edited.csv' # the path to the CSV file
save_json_path = 'annotations_coco.json'


data = pd.read_csv(path)

images = []
categories = []
annotations = []

category = {}
category["supercategory"] = 'none'
category["id"] = 0
category["name"] = 'None'
categories.append(category)

data['fileid'] = data['filename'].astype('category').cat.codes
data['categoryid']= pd.Categorical(data['class'],ordered= True).codes
data['categoryid'] = data['categoryid']+1
data['annid'] = data.index

def image(row):
    image = {}
    image["height"] = row.height
    image["width"] = row.width
    image["id"] = row.fileid
    image["file_name"] = row.filename
    return image

def category(row):
    category = {}
    category["supercategory"] = 'None'
    category["id"] = row.categoryid
    category["name"] = row[2]
    return category

def annotation(row):
    annotation = {}
    area = (row.xmax -row.xmin)*(row.ymax - row.ymin)
    annotation["segmentation"] = []
    annotation["iscrowd"] = 0
    annotation["area"] = area
    annotation["image_id"] = row.fileid

    annotation["bbox"] = [row.xmin, row.ymin, row.xmax -row.xmin,row.ymax-row.ymin ]

    annotation["category_id"] = row.categoryid
    annotation["id"] = row.annid
    return annotation

for row in data.itertuples():
    annotations.append(annotation(row))

imagedf = data.drop_duplicates(subset=['fileid']).sort_values(by='fileid')
for row in imagedf.itertuples():
    images.append(image(row))

catdf = data.drop_duplicates(subset=['categoryid']).sort_values(by='categoryid')
for row in catdf.itertuples():
    categories.append(category(row))

data_coco = {}
data_coco["images"] = images
data_coco["categories"] = categories
data_coco["annotations"] = annotations
json.dump(data_coco, open(save_json_path, "w"), indent=4)

print('Done!')

Done!
